# T01: Phaethon Fundamentals
Welcome to Phaethon! In this notebook, we will explore the core concepts of Type-Safe Physics, Dimensional Algebra, and how to build your own physical laws using the Axiom Engine.

In [15]:
# Setup the environment
try:
    import phaethon
    print(f"Phaethon version {phaethon.__version__} is ready to go!")
except ImportError:
    print("Installing Phaethon...")
    %pip install phaethon
    import phaethon
    print("Phaethon installed successfully!")

Phaethon version 0.2.0 is ready to go!


### 1. The Fluent API vs. Object-Oriented Units
Phaethon provides a quick chainable API for simple logging/UI tasks, and a robust Object-Oriented interface for serious math and arrays.

In [ ]:
import phaethon as ptn
from phaethon import u
from decimal import Decimal

# A. The Fluent API (Great for UI & Quick Scripts)
text_output = ptn.convert(100, 'km/h').to('m/s').use(format='verbose', delim=True).resolve()
print("Fluent API :", text_output)

# B. Object-Oriented Initialization
# Use floats (.mag) for Machine Learning, or Decimals (.exact) for Financial Audits
speed = u.KilometerPerHour(100)
print(f"OOP Float  : {speed.to(u.MeterPerSecond).mag} m/s")

gold_weight = u.Kilogram(Decimal('1.55'))
print(f"OOP Exact  : {gold_weight.to(u.Gram).exact} g")

Fluent API : 100 km/h = 27.777778 m/s
OOP Float  : 27.77777777777778 m/s
OOP Exact  : 1550.0 g


### 2. Metaclass Dimensional Algebra
You don't need to look up physics formulas. Phaethon's Metaclass engine automatically synthesizes new units when you multiply or divide them. Let's calculate Pressure (Force / Area).

In [ ]:
# Force: 50 Kips (Kilo-pounds)
applied_force = u.Kip(50.0)

# Area: 200 Square Inches
surface_area = u.SquareInch(200.0)

# Dimensional Algebra in action!
stress = applied_force / surface_area

print(f"Raw Stress  : {stress}")
print(f"Dimension   : {ptn.dimof(stress)}") # Phaethon knows this is Pressure!

# Convert the synthesized unit into standard Megapascals
print(f"Standardized: {stress.to(u.Megapascal).mag:.2f} MPa")

Raw Stress  : 0.25 ksi
Dimension   : pressure
Standardized: 1.72 MPa


### 3. The Axiom Engine (Building Custom Physics)
Missing a specific dimension? Build it! Let's synthesize `Carbon Intensity` (kgCO2 / kWh) and enforce a rule that emissions cannot be negative.

In [ ]:
from phaethon import axiom, BaseUnit

# 1. Define the Dimension & Bounds
@axiom.bound(min_val=0, msg="Emissions cannot be negative!")
class CarbonIntensity(BaseUnit):
    dimension = "carbon_intensity"

# 2. Derive the algebra
@axiom.derive(u.Kilogram / u.KilowattHour)
class KgCO2PerKWh(CarbonIntensity):
    symbol = "kgCO2/kWh"

# Let's test it!
co2_mass = u.Kilogram(5000)
energy = u.MegawattHour(10)

intensity = co2_mass / energy
print(f"Synthesized ESG Metric: {KgCO2PerKWh(intensity.mag).mag} {intensity.symbol}")

Synthesized ESG Metric: 500.0 kg/MWh


### 4. Schemas & Fields
The heart of Phaethon for Data Engineers is the `Schema`. It allows you to bind the strict physical rules we just learned directly into your Pandas workflows.

In [ ]:
import pandas as pd

# Let's create a declarative schema
class BasicLogistiptnSchema(ptn.Schema):
    # Enforce minimum boundaries and target dimensions declaratively
    cargo_weight: u.Kilogram = ptn.Field(
        source="raw_weight", 
        parse_string=True, 
        on_error='coerce', 
        min=0
    )

# A tiny dirty dataframe
df_mini = pd.DataFrame({
    'raw_weight': ["1500 lbs", "20 pallets", "-50 kg"]
})

print("--- RAW DATA ---")
display(df_mini)

# Normalize using vectorized execution
clean_df = BasicLogistiptnSchema.normalize(df_mini, keep_unmapped=True, drop_raw=True)

print("\n--- PHAETHON NORMALIZED DATA ---")
display(clean_df)
# Notice how "1500 lbs" converted to kg, "20 pallets" became NaN, and "-50 kg" violated the >0 bound and safely became NaN!

### 5. Algorithm Guardrails
Protect your domain algorithms from human error. We will force a function to ONLY accept Mass and Length, automatically converting them to floats for safe calculation.

In [13]:
# @require ensures only the correct dimensions enter.
# @prepare automatically extracts the .mag into standard base units.
@axiom.require(weight=u.MassUnit, distance=u.LengthUnit)
@axiom.prepare(weight=u.Kilogram, distance=u.Kilometer)
def calculate_shipping(weight: float, distance: float):
    # Safe to use standard Python math here
    return weight * distance * 0.05

cargo = u.Pound(500)
route = u.Mile(50)

cost = calculate_shipping(weight=cargo, distance=route)
print(f"Valid Shipping Cost: ${cost:,.2f}")

# Uncomment below to see the Guardrail trigger a TypeError!
# calculate_shipping(weight=u.UKGallon(50), distance=route)

Valid Shipping Cost: $912.48
